# Import the Libraries

In [ ]:
#Dataframe manipulation
import pandas as pd

#Track the progress of the loop
from tqdm import tqdm 

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim

hv.extension('bokeh')

#Librarie to save the plots to html object
import panel as pn

# Historical Data 

In [ ]:
#Import File with Data from Selected Tickers
info_df = pd.read_csv('Asset_Categories.csv')

ticker_list = info_df['Ticker'].to_list()

In [ ]:
#Create an empty dataframe and then run the loop to download the prices for all the categories
merged_prices_df = pd.DataFrame()

#Download the Data from Selected Tickers
for ticker in tqdm(ticker_list):

    category = info_df[info_df['Ticker'] == ticker]['Category'].iloc[0]

    #Download the Data from Yahoo Finance and reset the index
    tickers = Ticker(ticker)
    price_df = tickers.history(period = 'max')
    price_df = price_df.reset_index()

    #Select only the needed columns and change the name from Adjclose to the Category's Name
    price_df = price_df[['date','adjclose']]
    price_df = price_df.rename(columns = {'adjclose': category})

    #Convert the Date format to datetime
    price_df['date'] = pd.to_datetime(price_df['date'], utc=True)

    #Resample for Yearly
    price_df = price_df.resample('Y', on='date').last()

    #Set Index to Date for the index date to disappear
    price_df = price_df.set_index('date')

    #Calculate the Yearly Returns
    price_df = round(price_df.pct_change().dropna() * 100, 2)

    #Bring date from the Index and Change the format to show only date and not the date and time 
    price_df = price_df.reset_index()
    price_df['date'] = price_df['date'].dt.year

    #Merge the Dataframes in the loop
    if merged_prices_df.empty:
        merged_prices_df = price_df
    else:
        merged_prices_df = pd.merge(merged_prices_df, price_df, on='date', how='outer')
        
#Drop NA's
merged_prices_df = merged_prices_df.dropna()

#Create a column for the diversified (Equal weight)
merged_prices_df = merged_prices_df.set_index('date')
merged_prices_df['Diversified'] = round(merged_prices_df.mean(axis = 1), 2)

# Manipulate the Data to Create the Graph

In [ ]:
#Change the column name for Date to Year and set it as index
merged_prices_df = merged_prices_df.reset_index()
merged_prices_df = merged_prices_df.rename(columns = {'date': 'Year'})
merged_prices_df = merged_prices_df.set_index('Year')

#Transpose Dataframe
merged_prices_df = merged_prices_df.T

In [ ]:
#Create an empty dataframe and then run the loop to create a new dataframe based on the years
merged_sorted_df = pd.DataFrame()

#Run through the columns (years)
for column in merged_prices_df.columns:

    #Create a dataframe filtered for the Column in the loop
    sorted_df = pd.DataFrame(merged_prices_df[column])

    #Sort the Returns from higher to Lower
    sorted_df = sorted_df.sort_values(by=column, ascending=False)

    #Create a New column which will Include the Name of the Asset Category and the Returns
    sorted_df[column] = sorted_df.index + ': ' + sorted_df[column].apply(lambda x: '{:.2f}'.format(x))

    #Reset the Index for then to Include correctly in the Merged Sorted Dataframe
    sorted_df = sorted_df.set_index(column).reset_index()

    merged_sorted_df[column] = sorted_df

In [ ]:
#Change the Index to Start at 1 -> Looks nicer in the Table
merged_sorted_df.index = range(1, len(merged_sorted_df.index) + 1)

# Create the Chart with the Period Table of Investent Returns

In [ ]:
# Define a function to apply conditional formatting based on partial text
def highlight_partial_text(val):
    color_mapping = {
        'US Large Caps': 'red',
        'US Small Caps': 'navy',
        'World Developed': 'violet',
        'Emerging Markets': 'goldenrod',
        'US Fixed Income Aggregate': 'brown',
        'US High Yield': 'purple',
        'US TIPS': 'limegreen',
        'US T-Bill': 'gold',
        'International Treasury Bond': 'green',
        'Emerging Market Sovereign Bond': 'orange',
        'Commodities': 'gray',
        'Real Estate': 'darkturquoise',
        'Diversified': 'black'
    }
    
    for asset_class, color in color_mapping.items():
        if asset_class in val:
            return f'background-color: {color}; color: white'
    
    return ''

# Apply the function to the DataFrame using the Styler class
styled_df = merged_sorted_df.style.applymap(highlight_partial_text, subset=pd.IndexSlice[:, '2008':'2023'])

In [ ]:
# Convert the styled DataFrame to an HTML table
periodic_table_html = styled_df.to_html()

# Save the HTML table to a file
with open('Periodic_Table_Investments.html', 'w') as f:
    f.write(periodic_table_html)

# Create Table with Asset Class Mapping

In [ ]:
#Create Holoviews table object 
mapping_table = hv.Table(info_df).opts(
    opts.Table(width=600, height=250, selectable = True, index_position = None, 
               title = 'Asset Class Mapping Table'))

#Save the Plots
p = pn.panel(mapping_table)
p.save('Asset_Class_Mapping_Table.html', embed = True)